In [1]:
"""
1 GBM-GRB(GRB180423A) + 1 nonGBM-GRB(GRB190415A) = 2 GRBs (two different qsub)
6 time windows
46  different injections (0 ~ 9.1, step=0.2)
500 trials for each n_inj (run on 1 core)
10 s/trial
= 2k cpu hr = 1600 hivecpu * 2 hr

tws_in_second    = [  10,     25,    50,   100,   250,   500][::-1] 
"""

'\n1 GBM-GRB(GRB180423A) + 1 nonGBM-GRB(GRB190415A) = 2 GRBs (two different qsub)\n6 time windows\n46  different injections (0 ~ 9.1, step=0.2)\n500 trials for each n_inj (run on 1 core)\n10 s/trial\n= 2k cpu hr = 1600 hivecpu * 2 hr\n\ntws_in_second    = [  10,     25,    50,   100,   250,   500][::-1] \n'

In [2]:
import sys
import os
import glob
import numpy as np
import pandas as pd
sys.path.append('../../../')
from greco_grb.scripts.utils import *

In [3]:
from greco_grb.scripts import SETTING
paths = SETTING.PATH(osg=False)
print(paths)
LOCATION = paths.LOCATION
USER = paths.USER
ICDATA_DIR = paths.ICDATA_DIR
DATA_DIR = paths.DATA_DIR
ANA_DIR = paths.ANA_DIR


#### Working at GT ####

LOCATION: 	 GT
USER: 	 cchen641
ICDATA_DIR: 	 /storage/home/hhive1/cchen641/data/icecube/data/analyses/online_GRECO/candidate-002-p10
DATA_DIR: 	 /storage/home/hhive1/cchen641/data/icecube/data/greco_grb/data
ANA_DIR: 	 /storage/home/hhive1/cchen641/data/icecube/data/greco_grb/data/csky_output



In [4]:
### create arguments arrays ---> 03.txt
# 0: grb_name
# 1: n_inj
# 2: n_trials
# 3: tw_in_second
# 4: ncpu
# 5: batchIndex
# 6: use_poisson

# Check 1 out of 2

In [19]:
####### Check before running ########
script_path    = '/scratch/cjchen/submit/03.sub'
## [::-1] reverse order
grb_names      = ["GRB180423A"] # ["GRB180423A", "GRB170529A"]
n_injs         = np.round(np.r_[0:9.1:0.2], 2)
n_trials       = 500
tws_in_second  = [10, 25, 50, 100, 250, 500]  
ncpu           = 4
batchIndex     = 0
use_poisson    = 1
#######################################

In [20]:
contents = ""
for grb_name in grb_names:       
    for n_inj in n_injs:
        for tw_in_second in tws_in_second:
            contents += "{}\t".format(grb_name)
            contents += "{}\t".format(n_inj)
            contents += "{}\t".format(n_trials)
            contents += "{}\t".format(tw_in_second)
            contents += "{}\t".format(ncpu)
            contents += "{}\t".format(batchIndex)
            contents += "{}\n".format(use_poisson)

with open("03.txt", 'w') as writer:
    writer.write(contents)
    

In [21]:
# print(contents)

In [22]:
print("Total number of jobs: {}\n".format(contents.count('\n')))

Total number of jobs: 276



In [23]:
### create pbs file ---> 03.pbs

# Check 2 out of 2

In [24]:
nameofjob  = "03"
ntotal     = 276           # copy from the above 2nd cell "Total number of jobs: xxx"
inputtxt   = "03.txt"      # copy from the above 3rd cell
outdir     = ensure_dir("/storage/home/hhive1/cchen641/scratch/icecube/job_pbs_out/out")
walltimehr = 8 # 2hr is needed; 8 hrs for safe  # in Hour

In [25]:
contents = ""
contents += "#PBS -N {}\n".format(nameofjob)
contents += "#PBS -l nodes=1:ppn={}\n".format(1)           # number of nodes and cores per node required
contents += "#PBS -l pmem=4gb\n"                             # memory per core
contents += "#PBS -l walltime={}:00:00\n".format(walltimehr)  # dd:hh:mm:ss
contents += "#PBS -q hive\n"
contents += "#PBS -j oe\n"
contents += "#PBS -o {}\n".format(outdir+"/03.out")
contents += "#PBS -t 1-{}%400".format(ntotal)
contents += "\n"
contents += "cd /storage/home/hhive1/cchen641/icecube/greco_grb/scripts\n"
contents += "icpy3\n" 
# Get Paramters from input.txt file using $PBS_ARRAYID as the line number 
contents += 'params=`sed -n "${{PBS_ARRAYID}} p" {}`\n'.format("hive_sub/"+inputtxt)    # outer single quote to preserve inner double quotes
contents += "paramsArray=($params)\n"
contents += "arg0=${paramsArray[0]}\n"
contents += "arg1=${paramsArray[1]}\n"
contents += "arg2=${paramsArray[2]}\n"
contents += "arg3=${paramsArray[3]}\n"
contents += "arg4=${paramsArray[4]}\n"
contents += "arg5=${paramsArray[5]}\n"
contents += "arg6=${paramsArray[6]}\n"
contents += "python 03_do_signal_injection.py "

contents += "--grb_name $arg0 "
contents += "--n_inj $arg1 "
contents += "--n_trials $arg2 "
contents += "--tw_in_second $arg3 "
contents += "--ncpu $arg4 "
contents += "--batchIndex $arg5 "
contents += "--use_poisson $arg6 "
print(contents)
with open("03.pbs", 'w') as writer:
    writer.write(contents)

#PBS -N 03
#PBS -l nodes=1:ppn=1
#PBS -l pmem=4gb
#PBS -l walltime=8:00:00
#PBS -q hive
#PBS -j oe
#PBS -o /storage/home/hhive1/cchen641/scratch/icecube/job_pbs_out/out/03.out
#PBS -t 1-276%400
cd /storage/home/hhive1/cchen641/icecube/greco_grb/scripts
icpy3
params=`sed -n "${PBS_ARRAYID} p" hive_sub/03.txt`
paramsArray=($params)
arg0=${paramsArray[0]}
arg1=${paramsArray[1]}
arg2=${paramsArray[2]}
arg3=${paramsArray[3]}
arg4=${paramsArray[4]}
arg5=${paramsArray[5]}
arg6=${paramsArray[6]}
python 03_do_signal_injection.py --grb_name $arg0 --n_inj $arg1 --n_trials $arg2 --tw_in_second $arg3 --ncpu $arg4 --batchIndex $arg5 --use_poisson $arg6 
